In [1]:
import pandas as pd

In [2]:
pref = pd.read_csv("COVID-19-jp/nhk_news_covid19_prefectures_daily_data.csv")
pref["日付"] = pd.to_datetime(pref["日付"] )
print(len(pref))
pref.tail()

24158


,日付,都道府県コード,都道府県名,各地の感染者数_1日ごとの発表数,各地の感染者数_累計,各地の死者数_1日ごとの発表数,各地の死者数_累計
24153,2021-06-08,47,沖縄県,159,18762,0,154
24154,2021-06-09,47,沖縄県,174,18936,2,156
24155,2021-06-10,47,沖縄県,166,19102,0,156
24156,2021-06-11,47,沖縄県,145,19247,0,156
24157,2021-06-12,47,沖縄県,157,19404,3,159


In [3]:
latest_date = pref["日付"].max().strftime("%Y-%m-%d")

In [4]:
all_jp = pref.groupby(["都道府県名"], as_index=False).sum()
del all_jp["各地の感染者数_累計"]
del all_jp["各地の死者数_累計"]
#all_jp = all_jp.rename(columns={"各地の感染者数合計":"各地の感染者数_1日ごとの発表数"})
all_jp = all_jp.rename(columns={"各地の感染者数_1日ごとの発表数":"各地の感染者数合計"})
all_jp = all_jp.rename(columns={"各地の死者数_1日ごとの発表数":"各地の死者数合計"})
all_jp.tail()

,都道府県名,都道府県コード,各地の感染者数合計,各地の死者数合計
42,静岡県,11308,8831,143
43,香川県,19018,2062,26
44,高知県,20046,1652,21
45,鳥取県,15934,466,2
46,鹿児島県,23644,3569,33


## 都道府県ごとの最新データを付与

In [5]:
# 直近１週間の感染者数
pref["直近1週間の感染者数"] = pref["各地の感染者数_1日ごとの発表数"].rolling(7).sum()
pref.tail(7)

,日付,都道府県コード,都道府県名,各地の感染者数_1日ごとの発表数,各地の感染者数_累計,各地の死者数_1日ごとの発表数,各地の死者数_累計,直近1週間の感染者数
24151,2021-06-06,47,沖縄県,183,18499,0,153,1597.0
24152,2021-06-07,47,沖縄県,104,18603,1,154,1559.0
24153,2021-06-08,47,沖縄県,159,18762,0,154,1495.0
24154,2021-06-09,47,沖縄県,174,18936,2,156,1372.0
24155,2021-06-10,47,沖縄県,166,19102,0,156,1294.0
24156,2021-06-11,47,沖縄県,145,19247,0,156,1192.0
24157,2021-06-12,47,沖縄県,157,19404,3,159,1088.0


### 感染者数前日比

In [6]:
import numpy as np
pref["新規感染者数前日比"] = pref["各地の感染者数_1日ごとの発表数"].pct_change()
# 一番古い日付だけNANを設定
pref.loc[pref["日付"]==pref["日付"].min(),"新規感染者数前日比"] = np.nan
pref["直近1週間の新規感染者数前日比平均"] = pref["新規感染者数前日比"].rolling(7).mean()
pref["新規感染者数先週比"] = pref["各地の感染者数_1日ごとの発表数"].pct_change(periods=7)

pref.tail(10)

,日付,都道府県コード,都道府県名,各地の感染者数_1日ごとの発表数,各地の感染者数_累計,各地の死者数_1日ごとの発表数,各地の死者数_累計,直近1週間の感染者数,新規感染者数前日比,直近1週間の新規感染者数前日比平均,新規感染者数先週比
24148,2021-06-03,47,沖縄県,244,17808,1,153,1825.0,-0.178451,0.061601,0.016667
24149,2021-06-04,47,沖縄県,247,18055,0,153,1759.0,0.012295,0.019905,-0.210863
24150,2021-06-05,47,沖縄県,261,18316,0,153,1685.0,0.056680,0.017961,-0.220896
24151,2021-06-06,47,沖縄県,183,18499,0,153,1597.0,-0.298851,0.002560,-0.324723
24152,2021-06-07,47,沖縄県,104,18603,1,154,1559.0,-0.431694,0.008892,-0.267606
24153,2021-06-08,47,沖縄県,159,18762,0,154,1495.0,0.528846,0.002952,-0.286996
24154,2021-06-09,47,沖縄県,174,18936,2,156,1372.0,0.094340,-0.030976,-0.414141
24155,2021-06-10,47,沖縄県,166,19102,0,156,1294.0,-0.045977,-0.012052,-0.319672
24156,2021-06-11,47,沖縄県,145,19247,0,156,1192.0,-0.126506,-0.031880,-0.412955
24157,2021-06-12,47,沖縄県,157,19404,3,159,1088.0,0.082759,-0.028155,-0.398467


In [7]:
latest = pref.loc[pref["日付"]==pref["日付"].max()][["都道府県名","直近1週間の感染者数","新規感染者数前日比","直近1週間の新規感染者数前日比平均","新規感染者数先週比"]]
latest.head()

,都道府県名,直近1週間の感染者数,新規感染者数前日比,直近1週間の新規感染者数前日比平均,新規感染者数先週比
513,北海道,1081.0,-0.137931,-0.078593,-0.547101
1027,青森県,32.0,0.500000,0.319048,-0.666667
1541,岩手県,90.0,0.000000,0.238662,0.777778
2055,宮城県,59.0,0.500000,0.347006,-0.200000
2569,秋田県,2.0,inf,NaN,inf


In [8]:
all_jp = pd.merge(all_jp, latest, on="都道府県名")
all_jp.head()

,都道府県名,都道府県コード,各地の感染者数合計,各地の死者数合計,直近1週間の感染者数,新規感染者数前日比,直近1週間の新規感染者数前日比平均,新規感染者数先週比
0,三重県,12336,5126,108,93.0,-0.200000,0.013377,-0.478261
1,京都府,13364,16325,229,269.0,0.206897,0.366507,-0.285714
2,佐賀県,21074,2538,23,10.0,-1.000000,NaN,-1.000000
3,兵庫県,14392,40490,1256,345.0,-0.173913,0.093211,-0.530864
4,北海道,514,40390,1284,1081.0,-0.137931,-0.078593,-0.547101


## 人口データを付与

In [9]:
pop = pd.read_excel("COVID-19-jp/2001stjin.xls",header=3)[["Unnamed: 1","計"]]
pop.rename(columns={"Unnamed: 1":"都道府県名"},inplace=True)
pop.rename(columns={"計":"人口"},inplace=True)
pop = pop.loc[pop["都道府県名"]!="合計"].reset_index(drop=True)

In [10]:
all_jp = pd.merge(all_jp, pop, on="都道府県名", how="left")
all_jp.head()

,都道府県名,都道府県コード,各地の感染者数合計,各地の死者数合計,直近1週間の感染者数,新規感染者数前日比,直近1週間の新規感染者数前日比平均,新規感染者数先週比,人口
0,三重県,12336,5126,108,93.0,-0.200000,0.013377,-0.478261,1813859
1,京都府,13364,16325,229,269.0,0.206897,0.366507,-0.285714,2545899
2,佐賀県,21074,2538,23,10.0,-1.000000,NaN,-1.000000,823810
3,兵庫県,14392,40490,1256,345.0,-0.173913,0.093211,-0.530864,5549568
4,北海道,514,40390,1284,1081.0,-0.137931,-0.078593,-0.547101,5267762


## 人口当たりの感染者数

In [11]:
all_jp["１万人当たりの感染者数"] = all_jp["各地の感染者数合計"] / (all_jp["人口"] / 10000)
all_jp["10万人当たりの感染者数"] = all_jp["各地の感染者数合計"] / (all_jp["人口"] / 100000)
all_jp["直近1週間の10万人当たりの感染者数"] = all_jp["直近1週間の感染者数"] / (all_jp["人口"] / 100000)
all_jp.sort_values("都道府県コード")

,都道府県名,都道府県コード,各地の感染者数合計,各地の死者数合計,直近1週間の感染者数,新規感染者数前日比,直近1週間の新規感染者数前日比平均,新規感染者数先週比,人口,１万人当たりの感染者数,10万人当たりの感染者数,直近1週間の10万人当たりの感染者数
4,北海道,514,40390,1284,1081.0,-0.137931,-0.078593,-0.547101,5267762,76.673927,766.739272,20.521049
41,青森県,1028,2447,30,32.0,0.500000,0.319048,-0.666667,1275783,19.180378,191.803779,2.508264
19,岩手県,1542,1580,46,90.0,0.000000,0.238662,0.777778,1235517,12.788169,127.881688,7.284400
11,宮城県,2056,9023,84,59.0,0.500000,0.347006,-0.200000,2292385,39.360753,393.607531,2.573739
36,秋田県,2570,762,19,2.0,inf,NaN,inf,985416,7.732775,77.327748,0.202960
15,山形県,3084,2007,46,17.0,2.000000,NaN,-0.571429,1082296,18.543910,185.439104,1.570735
35,福島県,3598,4701,155,69.0,-0.666667,0.035559,-0.666667,1881981,24.978998,249.789982,3.666349
38,茨城県,4112,10111,157,220.0,4.600000,0.643490,2.111111,2921436,34.609692,346.096919,7.530543
27,栃木県,4626,6663,80,188.0,0.307692,0.106339,0.360000,1965516,33.899495,338.994951,9.564918
37,群馬県,5140,7976,148,64.0,-0.500000,0.243548,-0.400000,1969439,40.498843,404.988426,3.249656


## 感染者数

### 平均値、中央値

In [12]:
all_jp["各地の感染者数合計"].describe()

count        47.000000
mean      16396.425532
std       30018.688343
min         466.000000
25%        2492.500000
50%        4930.000000
75%       10652.000000
max      166065.000000
Name: 各地の感染者数合計, dtype: float64

### TOP 10

In [13]:
all_jp.sort_values("各地の感染者数合計", ascending=False).reset_index(drop=True).head(10)

,都道府県名,都道府県コード,各地の感染者数合計,各地の死者数合計,直近1週間の感染者数,新規感染者数前日比,直近1週間の新規感染者数前日比平均,新規感染者数先週比,人口,１万人当たりの感染者数,10万人当たりの感染者数,直近1週間の10万人当たりの感染者数
0,東京都,6682,166065,2150,2736.0,0.073563,0.042766,0.071101,13834925,120.033177,1200.331769,19.776038
1,大阪府,13878,101678,2524,968.0,-0.059701,0.083866,-0.275862,8849635,114.895134,1148.951341,10.938304
2,神奈川県,7196,64024,924,1459.0,0.122727,0.027422,0.102679,9209442,69.519956,695.199557,15.842436
3,愛知県,11822,49779,888,1126.0,-0.174497,-0.008019,-0.435780,7575530,65.710254,657.102539,14.863647
4,埼玉県,5654,44972,815,609.0,-0.045977,0.031729,-0.302521,7390054,60.854765,608.547651,8.240806
5,兵庫県,14392,40490,1256,345.0,-0.173913,0.093211,-0.530864,5549568,72.960634,729.606341,6.216700
6,北海道,514,40390,1284,1081.0,-0.137931,-0.078593,-0.547101,5267762,76.673927,766.739272,20.521049
7,千葉県,6168,38284,694,625.0,-0.300885,-0.040211,-0.423358,6319772,60.578135,605.781348,9.889597
8,福岡県,20560,34937,490,376.0,-0.189655,0.052550,-0.447059,5129841,68.105425,681.054247,7.329662
9,沖縄県,24158,19404,159,1088.0,0.082759,-0.028155,-0.398467,1481547,130.971208,1309.712078,73.436752


## １万人あたりの感染者数

### 平均値、中央値

In [14]:
all_jp["１万人当たりの感染者数"].describe()

count     47.000000
mean      39.528470
std       28.529691
min        7.732775
25%       21.997303
50%       30.115642
75%       51.656281
max      130.971208
Name: １万人当たりの感染者数, dtype: float64

## 直近１週間の１０万人あたりの感染者数

In [15]:
all_jp.sort_values(["直近1週間の10万人当たりの感染者数"],ascending=False).head(10)

,都道府県名,都道府県コード,各地の感染者数合計,各地の死者数合計,直近1週間の感染者数,新規感染者数前日比,直近1週間の新規感染者数前日比平均,新規感染者数先週比,人口,１万人当たりの感染者数,10万人当たりの感染者数,直近1週間の10万人当たりの感染者数
28,沖縄県,24158,19404,159,1088.0,0.082759,-0.028155,-0.398467,1481547,130.971208,1309.712078,73.436752
16,山梨県,9766,1832,20,188.0,-0.625000,0.204147,-0.680851,826579,22.163641,221.636407,22.744347
4,北海道,514,40390,1284,1081.0,-0.137931,-0.078593,-0.547101,5267762,76.673927,766.739272,20.521049
26,東京都,6682,166065,2150,2736.0,0.073563,0.042766,0.071101,13834925,120.033177,1200.331769,19.776038
32,神奈川県,7196,64024,924,1459.0,0.122727,0.027422,0.102679,9209442,69.519956,695.199557,15.842436
24,愛知県,11822,49779,888,1126.0,-0.174497,-0.008019,-0.435780,7575530,65.710254,657.102539,14.863647
44,高知県,20046,1652,21,80.0,-0.125000,-0.051281,-0.631579,709230,23.292867,232.928669,11.279839
9,大阪府,13878,101678,2524,968.0,-0.059701,0.083866,-0.275862,8849635,114.895134,1148.951341,10.938304
1,京都府,13364,16325,229,269.0,0.206897,0.366507,-0.285714,2545899,64.122732,641.227323,10.566012
5,千葉県,6168,38284,694,625.0,-0.300885,-0.040211,-0.423358,6319772,60.578135,605.781348,9.889597


## 直近１週間の新規感染者 増加速度

## 増加速度 統計情報

In [16]:
all_jp["直近1週間の新規感染者数前日比平均"].describe()

count    37.000000
mean      0.127435
std       0.183604
min      -0.078593
25%      -0.008188
50%       0.052550
75%       0.238662
max       0.643490
Name: 直近1週間の新規感染者数前日比平均, dtype: float64

### WORST 10(増加速度の早い順)

In [17]:
all_jp.sort_values("直近1週間の新規感染者数前日比平均",ascending=False).head(10)

,都道府県名,都道府県コード,各地の感染者数合計,各地の死者数合計,直近1週間の感染者数,新規感染者数前日比,直近1週間の新規感染者数前日比平均,新規感染者数先週比,人口,１万人当たりの感染者数,10万人当たりの感染者数,直近1週間の10万人当たりの感染者数
38,茨城県,4112,10111,157,220.0,4.600000,0.643490,2.111111,2921436,34.609692,346.096919,7.530543
6,和歌山県,15420,2651,47,16.0,2.000000,0.523810,0.500000,954258,27.780747,277.807469,1.676695
13,富山県,8224,1954,37,38.0,4.000000,0.492517,-0.444444,1055999,18.503805,185.038054,3.598488
25,新潟県,7710,3357,46,68.0,0.888889,0.369063,0.307692,2236042,15.013135,150.131348,3.041088
1,京都府,13364,16325,229,269.0,0.206897,0.366507,-0.285714,2545899,64.122732,641.227323,10.566012
11,宮城県,2056,9023,84,59.0,0.500000,0.347006,-0.200000,2292385,39.360753,393.607531,2.573739
41,青森県,1028,2447,30,32.0,0.500000,0.319048,-0.666667,1275783,19.180378,191.803779,2.508264
14,山口県,17990,3065,69,74.0,-0.461538,0.244250,-0.300000,1369882,22.374190,223.741899,5.401925
37,群馬県,5140,7976,148,64.0,-0.500000,0.243548,-0.400000,1969439,40.498843,404.988426,3.249656
19,岩手県,1542,1580,46,90.0,0.000000,0.238662,0.777778,1235517,12.788169,127.881688,7.284400


### BEST 10(増加速度の遅い順／減少速度の速い順)

In [18]:
all_jp.sort_values("直近1週間の新規感染者数前日比平均",ascending=True).head(10)

,都道府県名,都道府県コード,各地の感染者数合計,各地の死者数合計,直近1週間の感染者数,新規感染者数前日比,直近1週間の新規感染者数前日比平均,新規感染者数先週比,人口,１万人当たりの感染者数,10万人当たりの感染者数,直近1週間の10万人当たりの感染者数
4,北海道,514,40390,1284,1081.0,-0.137931,-0.078593,-0.547101,5267762,76.673927,766.739272,20.521049
31,石川県,8738,3869,111,45.0,-0.857143,-0.063946,-0.928571,1139612,33.950151,339.501515,3.948712
44,高知県,20046,1652,21,80.0,-0.125000,-0.051281,-0.631579,709230,23.292867,232.928669,11.279839
5,千葉県,6168,38284,694,625.0,-0.300885,-0.040211,-0.423358,6319772,60.578135,605.781348,9.889597
21,広島県,17476,11193,153,221.0,-0.029412,-0.038664,-0.297872,2826858,39.595197,395.951972,7.817867
18,岡山県,16962,7544,120,80.0,-0.083333,-0.038370,-0.450000,1903627,39.629612,396.296123,4.202504
29,滋賀県,12850,5398,83,117.0,0.083333,-0.036014,-0.617647,1420948,37.988723,379.887230,8.233940
42,静岡県,11308,8831,143,231.0,-0.325000,-0.033349,-0.448980,3708556,23.812503,238.125028,6.228839
28,沖縄県,24158,19404,159,1088.0,0.082759,-0.028155,-0.398467,1481547,130.971208,1309.712078,73.436752
17,岐阜県,10794,9075,176,169.0,0.409091,-0.008188,-0.279070,2032490,44.649666,446.496662,8.314924


## グラフ化

In [19]:
import matplotlib.pyplot as plt
%matplotlib notebook
#%matplotlib inline

In [20]:
all_jp_desc = all_jp.sort_values(["各地の感染者数合計"],ascending=False)
all_jp_desc_10k = all_jp.sort_values(["１万人当たりの感染者数"],ascending=False)
all_jp_desc_100k = all_jp.sort_values(["直近1週間の10万人当たりの感染者数"],ascending=False)

In [21]:
fig = plt.figure(figsize=(9.5,10))
plt.subplot(2,2,1) # (rows, columns, panel number)
plt.text(0,49,"※新型コロナ関連の情報提供:NHK ※人口の情報提供:e-Stat ※{}時点".format(latest_date),fontsize=7)
plt.title("各地の新規感染者数累計",y=0.95)
#plt.xticks(rotation="70",fontsize=9)
plt.yticks(fontsize=8)
plt.grid(axis="x")
chart_data = all_jp.sort_values(["各地の感染者数合計"])
#plt.bar(all_jp_desc["都道府県名"], all_jp_desc["各地の感染者数合計"])
plt.barh(chart_data["都道府県名"],chart_data["各地の感染者数合計"],align="center")
#plt.title("各地の感染者数", y=0.9)
#plt.text(5,-1,"※{}時点".format(latest_date),fontsize=9)

plt.subplot(2,2,2) # (rows, columns, panel number)
plt.title("1万人当たりの新規感染者数累計", y=0.95)
plt.yticks(fontsize=8)
plt.grid(axis="x")
chart_data = all_jp.sort_values(["１万人当たりの感染者数"])
plt.barh(chart_data["都道府県名"], chart_data["１万人当たりの感染者数"])
#plt.text(5,-1,"※{}時点".format(latest_date),fontsize=9)

plt.subplot(2,2,3) # (rows, columns, panel number)
plt.yticks(fontsize=8)
plt.grid(axis="x")
chart_data = all_jp.sort_values(["直近1週間の10万人当たりの感染者数"])
plt.barh(chart_data["都道府県名"], chart_data["直近1週間の10万人当たりの感染者数"])
plt.title("直近1週間の10万人当たりの新規感染者数",y=0.95)
# plt.text(0,chart_data["都道府県名"][-1],"{}時点".format(latest_date))
#plt.text(5,-1,"※{}時点".format(latest_date),fontsize=9)

### 新規感染者数の増加速度（直近１週間の前日比平均）
plt.subplot(2,2,4) # (rows, columns, panel number)
plt.title("新規感染者数前週同曜日比",y=0.95)
chart_data = all_jp.sort_values("新規感染者数先週比",na_position="first")
#chart_data = all_jp.sort_values("直近1週間の新規感染者数前日比平均",na_position="first")

plt.yticks(fontsize=8)
plt.grid(axis="x")
plt.barh(chart_data["都道府県名"], chart_data["新規感染者数先週比"])
#plt.text(chart_data["直近1週間の新規感染者数前日比平均"].max(),-1,"※{}時点".format(latest_date),fontsize=9,ha="right")

#fig.text(0,0,"※{}時点".format(latest_date),fontsize=9)

fig.tight_layout()


<IPython.core.display.Javascript object>

In [22]:
fig.savefig("out/covid-domestic.png")

## 度数分布表

In [23]:
import numpy as np
data = np.array(all_jp["各地の感染者数合計"])
 
# ヒストグラム
hist, bin_edges = np.histogram(data, bins=8)

print(hist)
print(bin_edges)

[38  4  2  1  1  0  0  1]
[   466.     21165.875  41865.75   62565.625  83265.5   103965.375
 124665.25  145365.125 166065.   ]


## ヒストグラム

In [24]:
fig = plt.figure(figsize=(10,3))
plt.hist(all_jp_desc["各地の感染者数合計"],bins=[0,20000,40000,80000,100000,120000,140000,160000])

<IPython.core.display.Javascript object>

(array([38.,  2.,  5.,  0.,  1.,  0.,  0.]),
 array([     0,  20000,  40000,  80000, 100000, 120000, 140000, 160000]),
 <a list of 7 Patch objects>)

In [25]:
fig = plt.figure(figsize=(10,3))
plt.hist(all_jp_desc["10万人当たりの感染者数"],bins=[0,200,400,600,800,1000,1200])


<IPython.core.display.Javascript object>

(array([10., 23.,  3.,  8.,  0.,  1.]),
 array([   0,  200,  400,  600,  800, 1000, 1200]),
 <a list of 6 Patch objects>)

In [26]:
pref[pref["都道府県名"]=="和歌山県"].tail(10)


,日付,都道府県コード,都道府県名,各地の感染者数_1日ごとの発表数,各地の感染者数_累計,各地の死者数_1日ごとの発表数,各地の死者数_累計,直近1週間の感染者数,新規感染者数前日比,直近1週間の新規感染者数前日比平均,新規感染者数先週比
15410,2021-06-03,30,和歌山県,4,2633,0,45,39.0,-0.500000,2.397969,-0.500000
15411,2021-06-04,30,和歌山県,0,2633,0,45,38.0,-1.000000,2.380112,-1.000000
15412,2021-06-05,30,和歌山県,2,2635,0,45,23.0,inf,NaN,-0.882353
15413,2021-06-06,30,和歌山県,2,2637,0,45,20.0,0.000000,NaN,-0.600000
15414,2021-06-07,30,和歌山県,1,2638,0,45,20.0,-0.500000,NaN,0.000000
15415,2021-06-08,30,和歌山県,1,2639,0,45,18.0,0.000000,NaN,-0.666667
15416,2021-06-09,30,和歌山県,2,2641,2,47,12.0,1.000000,NaN,-0.750000
15417,2021-06-10,30,和歌山県,6,2647,0,47,14.0,2.000000,NaN,0.500000
15418,2021-06-11,30,和歌山県,1,2648,0,47,15.0,-0.833333,NaN,inf
15419,2021-06-12,30,和歌山県,3,2651,0,47,16.0,2.000000,0.523810,0.500000
